In [1]:
# STUDENT_NAME = "xxx" #Put your name
# STUDENT_ROLLNO = "xxx" #Put your roll number
# CODE_COMPLETE = True
# set the above to True if you were able to complete the code
# and that you feel your model can generate a good result
# otherwise keep it as False
# Don't lie about this. This is so that we don't waste time with
# the autograder and just perform a manual check
# If the flag above is True and your code crashes, that's
# an instant deduction of 2 points on the assignment.
#
#@PROTECTED_1_BEGIN
## No code within "PROTECTED" can be modified.
## We expect this part to be VERBATIM.
## IMPORTS 
## No other library imports other than the below are allowed.
## No, not even Scipy
import numpy as np 
import pandas as pd 
import sklearn.model_selection as model_selection 
import sklearn.preprocessing as preprocessing 
import sklearn.metrics as metrics 
from tqdm import tqdm # You can make lovely progress bars using this

## FILE READING: 
## You are not permitted to read any files other than the ones given below.
X_train = pd.read_csv("train_X.csv",index_col=0).to_numpy()
y_train = pd.read_csv("train_y.csv",index_col=0).to_numpy().reshape(-1,)
X_test = pd.read_csv("test_X.csv",index_col=0).to_numpy()
# submissions_df = pd.read_csv("sample_submission.csv",index_col=0)
#@PROTECTED_1_END

#protected_2 has been moved to the end UNEDITED

In [2]:
X_train = X_train/255.0
X_test = X_test/255.0

In [3]:
def set_params():
    weight_1 = np.random.randn(1024,784)*np.sqrt(1.0/1024)
    bias_1 = np.random.randn(1024,1)*np.sqrt(1.0/1024)
    weight_2 = np.random.randn(512,1024)*np.sqrt(1.0/512)
    bias_2 = np.random.randn(512,1)*np.sqrt(1.0/512)
    weight_3 = np.random.randn(256,512)*np.sqrt(1.0/256)
    bias_3 = np.random.randn(256,1)*np.sqrt(1.0/256)
    weight_4 = np.random.randn(10,256)*np.sqrt(1.0/10)
    bias_4 = np.random.randn(10,1)*np.sqrt(1.0/10)
    return weight_1,bias_1,weight_2,bias_2,weight_3,bias_3,weight_4,bias_4

In [4]:
def RELU(Z):
    return np.maximum(Z,0)

def softmax(Z):
    return np.exp(Z)/sum(np.exp(Z))

In [5]:
def d_RELU(Z):
    return Z > 0

In [6]:
def ForwardPropagation(weight_1, bias_1, weight_2, bias_2, weight_3, bias_3, weight_4, bias_4, X):
#     print (weight_1.shape)
#     print (X.shape)
#     print (bias_1.shape)
    
    Z1 = weight_1.dot(X) + bias_1
    activation_layer_1 = RELU(Z1)

    Z2 = weight_2.dot(activation_layer_1) + bias_2
    activation_layer_2 = RELU(Z2)

    Z3 = weight_3.dot(activation_layer_2) + bias_3
    activation_layer_3 = RELU(Z3)

    Z4 = weight_4.dot(activation_layer_3) + bias_4
    activation_layer_4 = softmax(Z4)

    return  Z1, activation_layer_1, Z2, activation_layer_2, Z3, activation_layer_3, Z4, activation_layer_4

In [7]:
def OneHotEncoding(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

In [8]:
def BackwardPropagation(Z1, activation_layer_1, Z2, activation_layer_2, Z3, activation_layer_3, Z4, activation_layer_4, weight_1, weight_2, 
                        weight_3, weight_4, X, Y):
    one_hot_Y = OneHotEncoding(Y)
    d_Z4 = activation_layer_4 - one_hot_Y

    d_weight_4 = d_Z4.dot(activation_layer_3.T)/60000
    d_bias_4 = np.sum(d_Z4)/60000
    d_Z3 = weight_4.T.dot(d_Z4)*d_RELU(Z3)

    d_weight_3 = d_Z3.dot(activation_layer_2.T)/60000
    d_bias_3 = np.sum(d_Z3)/60000
    d_Z2 = weight_3.T.dot(d_Z3)*d_RELU(Z2)

    d_weight_2 = d_Z2.dot(activation_layer_1.T)/60000
    d_bias_2 = np.sum(d_Z2)/60000
    d_Z1 = weight_2.T.dot(d_Z2)*d_RELU(Z1)

    d_weight_1 = d_Z1.dot(X.T)/60000
    d_bias_1 = np.sum(d_Z1)/60000
    
    return  d_weight_1, d_bias_1, d_weight_2,d_bias_2, d_weight_3, d_bias_3, d_weight_4, d_bias_4

In [9]:
def update_params(weight_1, bias_1, weight_2, bias_2, weight_3, bias_3, weight_4, bias_4, d_weight_1, d_bias_1, d_weight_2, d_bias_2, d_weight_3, d_bias_3, d_weight_4, d_bias_4, learning_rate):
    weight_1 = weight_1 - (learning_rate * d_weight_1)
    bias_1 = bias_1 - (learning_rate * d_bias_1)   
    weight_2 = weight_2 - (learning_rate * d_weight_2)  
    bias_2 = bias_2 - (learning_rate * d_bias_2)
    weight_3 = weight_3 - (learning_rate * d_weight_3)  
    bias_3 = bias_3 - (learning_rate * d_bias_3)
    weight_4 = weight_4 - (learning_rate * d_weight_4)  
    bias_4 = bias_4 - (learning_rate * d_bias_4)
    return  weight_1, bias_1, weight_2,  bias_2, weight_3, bias_3, weight_4, bias_4

In [10]:
def predict(X):
    return np.argmax(X, 0)

In [11]:
def predict_acc(scores, Y):
    return np.sum(scores == Y) / Y.size

In [12]:
def gradient_descent(X, Y, learning_rate, epochs):
    weight_1, bias_1, weight_2, bias_2, weight_3, bias_3, weight_4, bias_4 = set_params()
    for i in tqdm(range(epochs)):
        Z1, activation_layer_1, Z2, activation_layer_2, Z3, activation_layer_3, Z4, activation_layer_4 = ForwardPropagation(weight_1, bias_1, weight_2, bias_2, weight_3, bias_3, weight_4, bias_4, X.T)
        d_weight_1, d_bias_1, d_weight_2, d_bias_2, d_weight_3, d_bias_3, d_weight_4, d_bias_4 = BackwardPropagation(Z1, activation_layer_1, 
                                                                                                               Z2, activation_layer_2, 
                                                                                                               Z3, activation_layer_3, 
                                                                                                               Z4, activation_layer_4, 
                                                                                                               weight_1, weight_2, weight_3, 
                                                                                                               weight_4, X.T, Y.T)
        weight_1, bias_1, weight_2, bias_2, weight_3, bias_3, weight_4, bias_4 = update_params(weight_1, bias_1, weight_2, bias_2, 
                                                                                              weight_3, bias_3, weight_4, bias_4, 
                                                                                              d_weight_1, d_bias_1, d_weight_2, 
                                                                                              d_bias_2, d_weight_3, d_bias_3, 
                                                                                              d_weight_4, d_bias_4, learning_rate)
    return  weight_1, bias_1, weight_2, bias_2, weight_3, bias_3, weight_4, bias_4

In [13]:
weight_1, bias_1, weight_2, bias_2, weight_3, bias_3, weight_4, bias_4 = gradient_descent(X_train, y_train, 0.1,210)

100%|██████████| 210/210 [21:35<00:00,  6.17s/it]


In [30]:
def give_pd(X, weight_1, bias_1, weight_2, bias_2, weight_3, bias_3, weight_4, bias_4):
    _, _, _, _, _, _ ,_ ,activation_layer_4 = ForwardPropagation(weight_1, bias_1, weight_2, bias_2, weight_3, bias_3, weight_4, bias_4, X.T)
    scores = predict(activation_layer_4)
    print(activation_layer_4.shape)
    return scores

In [31]:
variable_ = give_pd(X_train, weight_1, bias_1, weight_2, bias_2, weight_3, bias_3, weight_4, bias_4)
print(predict_acc(variable_,y_train))

(10, 60000)
0.9652333333333334


In [32]:
final_op = give_pd(X_test,weight_1, bias_1, weight_2, bias_2,weight_3, bias_3, weight_4, bias_4)

(10, 10000)


In [33]:
submissions_df['target'] = final_op

In [34]:
# #@PROTECTED_2_BEGIN 
# ##FILE WRITING:
# # You are not permitted to write to any file other than the one given below.
# submissions_df.to_csv("{}__{}.csv".format(STUDENT_ROLLNO,STUDENT_NAME))
# #@PROTECTED_2_END